In [11]:
import os
import boto3
import pyspark.sql
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()

# Set-up bucket and spark

In [12]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.5.jar pyspark-shell"

Set-up spark session and context.

In [13]:
spark = SparkSession \
    .builder \
    .appName("computer_vision") \
    .master("local[*]") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
    .config ("spark.hadoop.fs.gs.project.id", "project_id") \
    .config("spark.hadoop.fs.gs.auth.service.account.email", "mail") \
    .config("spark.hadoop.fs.gs.auth.service.account.private.key.id", "key_id") \
    .config("spark.hadoop.fs.gs.auth.service.account.private.key", "private_key") \
    .getOrCreate()

img/
img/apple_crimson_snow/
img/apple_crimson_snow/0_100.jpg
img/apple_crimson_snow/177_100.jpg
img/banana/
img/banana/0_100.jpg
img/banana/41_100.jpg


In [ ]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

Check bucket content.

In [ ]:
!gsutil ls gs://p8_computer_vision/img/*

Import data from bucket.

In [17]:
path_img = "gs://p8_computer_vision/img/*"

img_all = spark.read.format("binaryfile").load(path_img)
img_all.printSchema()

Py4JJavaError: An error occurred while calling o55.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2197)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2654)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:376)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.sc